In [3]:
import geopandas as gpd 
import matplotlib.pyplot as plt
import fiona 
import numpy as np
import pandas as pd 
from shapely.geometry import LineString, Point, MultiPoint
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle

In [20]:
# READ KML file to a geopandas dataframe 
fiona.drvsupport.supported_drivers['KML'] = 'rw'

geo_df = gpd.read_file('edav comm cont/20231029-170354 - Edav 1.kml',driver='KML')
geo_df.head()

,Name,Description,geometry
0,Track 20231029-170354,<b>Edav 1</b><br><br>Distance = 5.16 km<br>Dur...,"LINESTRING Z (-73.94829 40.79047 -17.13900, -7..."


In [21]:
# Converting Shapely Lines to Points
points = [Point(coord) for coord in geo_df['geometry'].values[0].coords]
gdf = gpd.GeoDataFrame(geometry=points)
gdf.head()

,geometry
0,POINT Z (-73.94829 40.79047 -17.13900)
1,POINT Z (-73.94829 40.79046 -17.13900)
2,POINT Z (-73.94831 40.79045 -17.81700)
3,POINT Z (-73.94832 40.79046 -18.19800)
4,POINT Z (-73.94833 40.79047 -18.40400)


In [22]:
# Setting coordinate system and plotting interactive plot
gdf = gdf.set_crs('EPSG:4326')
gdf.explore(tiles='CartoDB Positron')

In [23]:
# extracting only the latitude and longitude from shapely point
df = gdf.apply(lambda row: pd.Series({'lon': row['geometry'].x,
                                      'lat': row['geometry'].y}), 
               axis=1)

df.head()

,lon,lat
0,-73.948293,40.790469
1,-73.948291,40.790461
2,-73.948313,40.790446
3,-73.948324,40.790460
4,-73.948325,40.790468


In [24]:
# Calculating cluster labels for each data point
coords = np.array(df[['lat','lon']]).reshape(-1,2) 

kms_per_radian = 6371.0088
epsilon = 0.0025 / kms_per_radian

db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine') 
db.fit(np.radians(coords))
cluster_labels = db.labels_

num_clusters = len(set(cluster_labels))
clusters = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))


Number of clusters: 318


In [25]:
# finding the closest datapoint to the cluster's center
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)
centermost_points = clusters.map(get_centermost_point)

In [26]:
# filtering out datapoints from original dataframe that is closest to cluster center
lats, lons = zip(*centermost_points)
rep_points = pd.DataFrame({'lon':lons, 'lat':lats})

rs = rep_points.apply(lambda row: df[(df['lat']==row['lat']) &
                                     (df['lon']==row['lon'])].iloc[0], 
                      axis=1)
rs

,lon,lat
0,-73.948431,40.790478
1,-73.948220,40.790464
2,-73.948177,40.790443
3,-73.948449,40.790416
4,-73.948619,40.790364
...,...,...
313,-73.948851,40.790563
314,-73.948764,40.790637
315,-73.948675,40.790531
316,-73.948423,40.790401


In [27]:
# Converting the coordinates to shapely points to create geodataframe for plotting
geometry = [Point(lon, lat) for lon, lat in zip(rs['lon'], rs['lat'])]
gdf_small = gpd.GeoDataFrame(rs, geometry=geometry)
gdf_small.head()

,lon,lat,geometry
0,-73.948431,40.790478,POINT (-73.94843 40.79048)
1,-73.948220,40.790464,POINT (-73.94822 40.79046)
2,-73.948177,40.790443,POINT (-73.94818 40.79044)
3,-73.948449,40.790416,POINT (-73.94845 40.79042)
4,-73.948619,40.790364,POINT (-73.94862 40.79036)


In [28]:
# Setting coordinate system to map points onto and plotting interactive plot
gdf_small = gdf_small.set_crs('EPSG:4326')
gdf_small.explore(tiles='CartoDB Positron')